![alt text](https://static.wixstatic.com/media/f11c7c_69283cb47c30496d953e1d8d4c6b5b18~mv2.png/v1/fill/w_75,h_75,al_c,usm_0.66_1.00_0.01/Logo%20new.png)


# IBM Data Science capstone project
## The battle of the neighborhoods

This notebook holds my final assignment for the IBM Data Science certificate.

---

In [12]:
# import the modules needed
import pandas as pd
import numpy as np

In [2]:
# the first line as requested in the assignment
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Author
<a href="https://www.linkedin.com/in/mjellema/" target="_blank">Marc Jellema</a>



The above cell where the end of the prequel for the capstone project. The cells below this point contain the real work.

In [160]:
# extra environment setup
# library to retrieve data from CBS StatLine
!pip install cbsodata
import cbsodata

# let's make tapping into foursquare a bit easier
!pip install foursquare
import foursquare

# library to handle JSON files
import json

# tranform JSON file into a pandas dataframe
from pandas import json_normalize 

# library for issueing requests
import requests

print()
print('Libraries imported and ready to use')

You should consider upgrading via the '/Users/m.jellema/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/m.jellema/opt/anaconda3/bin/python -m pip install --upgrade pip' command.

Libraries imported and ready to use


In [161]:
# fetch all tables from the CBS data set
cbstables = cbsodata.get_table_list()
print()
print('CBS tables fetched from StatLine')


CBS tables fetched from StatLine


In [162]:
# prepare the inhabitants table from the data retrieved from CBS StatLine
# filter on the table Title to get the right table
cbs_tbl1 = list(filter(lambda table1: table1['Title'] == 'Regionale kerncijfers Nederland', cbstables))

# use the identifier to fill the full data frame
inhabitants = pd.DataFrame(cbsodata.get_data(cbs_tbl1[0]['Identifier']))

# skip unnecessary columns & rows
inhabitants_subset = inhabitants[['RegioS','Perioden','TotaleBevolking_1']]
inhabitants_subset = inhabitants_subset.loc[(inhabitants_subset['Perioden']=='2019') & \
                                           (inhabitants_subset['RegioS'].str.contains("PV"))]

# prepare the merge data frame
inhabitants_subset["RegioS"] = inhabitants_subset["RegioS"].str.replace("\(PV\)", "")
inhabitants_subset["RegioS"] = inhabitants_subset["RegioS"].str.replace("Fryslân", "Friesland")
inhabitants_subset.rename(columns={'RegioS': 'Province', 'TotaleBevolking_1': 'Inhabitants'}, inplace=True)
inhabitants_merge_set = inhabitants_subset[['Province','Inhabitants']]
inhabitants_merge_set

,Province,Inhabitants
154,Groningen,583990.0
180,Friesland,647672.0
206,Drenthe,492167.0
232,Overijssel,1156431.0
258,Flevoland,416546.0
284,Gelderland,2071972.0
310,Utrecht,1342158.0
336,Noord-Holland,2853359.0
362,Zuid-Holland,3673893.0
388,Zeeland,383032.0


In [163]:
# prepare the average price table from the data retrieved from CBS StatLine
# filter on the table Title to get the right table
cbs_tbl2 = list(filter(lambda table1: table1['Title'] == 'Bestaande koopwoningen; gemiddelde verkoopprijzen, regio', cbstables))

# use the identifier to fill the full data frame
avgPrices = pd.DataFrame(cbsodata.get_data(cbs_tbl2[0]['Identifier']))

# skip unnecessary columns & rows
avgPrices_subset = avgPrices[['RegioS','Perioden','GemiddeldeVerkoopprijs_1']]
avgPrices_subset = avgPrices_subset.loc[(avgPrices_subset['Perioden']=='2019') & \
                                        (avgPrices_subset['RegioS'].str.contains("PV"))]

# prepare the merge data frame
avgPrices_subset["RegioS"] = avgPrices_subset["RegioS"].str.replace("\(PV\)", "")
avgPrices_subset.rename(columns={'RegioS': 'Province', 'GemiddeldeVerkoopprijs_1': 'AvgPrice'}, inplace=True)
avgPrices_merge_set = avgPrices_subset[['Province','AvgPrice']]
avgPrices_merge_set

,Province,AvgPrice
149,Groningen,219283.0
174,Friesland,230643.0
199,Drenthe,241941.0
224,Overijssel,260130.0
249,Flevoland,269589.0
274,Gelderland,296243.0
299,Utrecht,371727.0
324,Noord-Holland,396601.0
349,Zuid-Holland,305261.0
374,Zeeland,242998.0


In [164]:
# let's setup the necessary FourSquare API call parts
CLIENT_ID = 'MQVZNPQRDHH2QCAIZKHHD1UGMRHET0L1FGGPHXKPZ2PNTQTP' # your Foursquare ID
CLIENT_SECRET = 'Y5YLRS450FQS0M1K02QAGGYEFESWYT0XYCYM2UHKLOIFS0QD' # your Foursquare Secret
VERSION = '20200616' # Foursquare API version
URL = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, 500)


In [166]:
# Let's fetch the Foursquare data
# List of province capitals in the Netherlands
# prepare a dictionary of province name: capital
provinces = {
    'Groningen' : 'Groningen', 
    'Friesland':'Leeuwarden', 
    'Drenthe':'Assen',
    'Overijssel': 'Zwolle', 
    'Flevoland':'Lelystad', 
    'Gelderland': 'Arnhem',
    'Utrecht':'Utrecht', 
    'Noord-Holland':'Amsterdam', 
    'Zuid-Holland':'Den Haag',
    'Zeeland': 'Middelburg', 
    'Noord-Brabant':"'s-Hertogenbosch", 
    'Limburg':'Maastricht'
}

# Thanks for G. Bareirro for providing this handy list of cuisines and their ID's
cuisines = {
    'Afghan': '503288ae91d4c4b30a586d67', 
    'African': '4bf58dd8d48988d1c8941735', 
    'American': '4bf58dd8d48988d14e941735', 
    'Burmese': '56aa371be4b08b9a8d573568', 
    'Cambodian': '52e81612bcbc57f1066b7a03', 
    'Chinese': '4bf58dd8d48988d145941735', 
    'Filipino': '4eb1bd1c3b7b55596b4a748f', 
    'Himalayan': '52e81612bcbc57f1066b79fb', 
    'Indonesian': '4deefc054765f83613cdba6f', 
    'Japanese': '4bf58dd8d48988d111941735', 
    'Korean': '4bf58dd8d48988d113941735', 
    'Malay': '4bf58dd8d48988d156941735', 
    'Mongolian': '4eb1d5724b900d56c88a45fe', 
    'Thai': '4bf58dd8d48988d149941735', 
    'Tibetan': '52af39fb3cf9994f4e043be9', 
    'Vietnamese': '4bf58dd8d48988d14a941735', 
    'Australian': '4bf58dd8d48988d169941735', 
    'Austrian': '52e81612bcbc57f1066b7a01', 
    'Bangladeshi': '5e179ee74ae8e90006e9a746', 
    'Belgian': '52e81612bcbc57f1066b7a02', 
    'Caribbean': '4bf58dd8d48988d144941735', 
    'Caucasian': '5293a7d53cf9994f4e043a45', 
    'Czech': '52f2ae52bcbc57f1066b8b81', 
    'Dutch': '5744ccdfe4b0c0459246b4d0', 
    'Belarusian': '52e928d0bcbc57f1066b7e97', 
    'Bosnian': '58daa1558bbb0b01f18ec1ee', 
    'Bulgarian': '56aa371be4b08b9a8d5734f3', 
    'Romanian': '52960bac3cf9994f4e043ac4', 
    'Tatar': '52e928d0bcbc57f1066b7e98', 
    'English': '52e81612bcbc57f1066b7a05', 
    'French': '4bf58dd8d48988d10c941735', 
    'German': '4bf58dd8d48988d10d941735', 
    'Greek': '4bf58dd8d48988d10e941735', 
    'Hawaiian': '52e81612bcbc57f1066b79fe', 
    'Hungarian': '52e81612bcbc57f1066b79fa', 
    'Indian': '4bf58dd8d48988d10f941735', 
    'Italian': '4bf58dd8d48988d110941735', 
    'Salvadoran': '5745c7ac498e5d0483112fdb', 
    'Argentinian': '4bf58dd8d48988d107941735', 
    'Brazilian': '4bf58dd8d48988d16b941735', 
    'Colombian': '58daa1558bbb0b01f18ec1f4', 
    'Peruvian': '4eb1bfa43b7b52c0e1adc2e8', 
    'Venezuelan': '56aa371be4b08b9a8d573558', 
    'Moroccan': '4bf58dd8d48988d1c3941735', 
    'Mexican': '4bf58dd8d48988d1c1941735', 
    'Egyptian': '5bae9231bedf3950379f89e1', 
    'Iraqi': '5bae9231bedf3950379f89e7', 
    'Israeli': '56aa371be4b08b9a8d573529', 
    'Kurdish': '5744ccdfe4b0c0459246b4ca', 
    'Lebanese': '58daa1558bbb0b01f18ec1cd', 
    'Persian': '52e81612bcbc57f1066b79f7', 
    'Syrian': '5bae9231bedf3950379f89da', 
    'Yemeni': '5bae9231bedf3950379f89ea', 
    'Pakistani': '52e81612bcbc57f1066b79f8', 
    'Polish': '52e81612bcbc57f1066b7a04', 
    'Portuguese': '4def73e84765ae376e57713a', 
    'Russian': '5293a7563cf9994f4e043a44', 
    'Scandinavian': '4bf58dd8d48988d1c6941735', 
    'Scottish': '5744ccdde4b0c0459246b4a3', 
    'Slovak': '56aa371be4b08b9a8d57355a', 
    'Spanish': '4bf58dd8d48988d150941735', 
    'Sri Lankan': '5413605de4b0ae91d18581a9', 
    'Swiss': '4bf58dd8d48988d158941735', 
    'Turkish': '4f04af1f2fb6e1c99f3db0bb', 
    'Ukranian': '52e928d0bcbc57f1066b7e96'
}

# prepare the data frame for the restaurants 
restaurants = pd.DataFrame(columns=['Province', 'Cuisine','Number of restaurants'])

# loop through the province and cuisines dictionaries and store the resuls
for province, capital in provinces.items():
    print(province)
    for cuisine, category_id in cuisines.items():
        # For each capital, retrieve the number of restaurants from foursquare
        url = "{url}&near={capital}&categoryId={category_id}".format(url=URL, capital=capital+', the Netherlands', category_id=category_id)
        query = requests.get(url)
        if query.status_code == 200:
            results = query.json()
            venues = results['response']['groups'][0]['items']
            restaurants = restaurants.append({
                'Province': province,
                'Cuisine': cuisine, 
                'Number of restaurants': len(venues)
            }, ignore_index=True)
restaurants.shape

Groningen
Friesland
Drenthe
Overijssel
Flevoland
Gelderland
Utrecht
Noord-Holland
Zuid-Holland
Zeeland
Noord-Brabant
Limburg


(780, 3)

In [167]:
# group and sum the restaurants
restaurants_city = restaurants[['Province','Number of restaurants']].groupby('Province').sum().reset_index()
restaurants_city

,Province,Number of restaurants
0,Drenthe,80
1,Flevoland,226
2,Friesland,106
3,Gelderland,260
4,Groningen,180
5,Limburg,279
6,Noord-Brabant,286
7,Noord-Holland,1395
8,Overijssel,180
9,Utrecht,381
